In [77]:
import numpy as np
import pandas as pd
from dataset import GetData
from nilearn.maskers import NiftiMasker
from permute_labels import PermLabels
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC

# Set directories

In [78]:
DATADIR = "/home/emanuele/Documents/bnb_retreat/data/sub-01/"
OUTPUT = "searchlight_output"
MASK = "/home/emanuele/Documents/bnb_retreat/data/masks/binarized_ofc.nii.gz"

# Set the necessary variables

In [79]:
TASKS = ["col", "pl"]
SUBJ = "01"

# Get dataset

In [80]:
dataset = GetData(
    tasks=TASKS, labels={"_f_": 1, "_nf_": 2}, group_filter=TASKS
)
data = dataset(DATADIR)
print(data)

{'data': array(['/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_3_run-3_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_1_run-2_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_col_block_1_run-2_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_col_block_1_run-1_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_2_run-2_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_1_run-1_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_col_block_2_run-2_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_2_run-1_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_1_run-3_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_col_block_3_run-3_zmap.nii.gz',
       

# Let's take advantage of the Nilearn masker
In this case we use it only to mask our samples, but it has several useful functionalities, check it out!

We are using a very coarse and big OFC mask (only becuase that's what we have available), of course we should use a brain parcellation to run a more meaningful ROI analysis.

In [81]:
MASKER = NiftiMasker(mask_img=MASK, memory="nilearn_cache", memory_level=1)
FMRI_MASKED = MASKER.fit_transform(data["data"])

# Set up the crossvalidation

In [82]:
CV = LeaveOneGroupOut()

# We set a Linear support vector classifier using the parameters suggested by the Nilearn decoder

In [83]:
CLF = LinearSVC(penalty="l1", dual=False, max_iter=1e4)

# Let's run the classification using cross_val_score from sk_learn

In [84]:
cv_scores = cross_val_score(
    CLF,
    X=FMRI_MASKED,
    y=data["labels"],
    cv=CV,
    scoring="accuracy",
    groups=data["groups"],
    n_jobs=-1,
)
print(f"folds scores {cv_scores}", f"average scores {np.mean(cv_scores)}")

folds scores [0.66666667 0.44444444] average scores 0.5555555555555556


## Again we would like to run a permutation test, therefore we get our chance maps

In [85]:
CV = list(CV.split(data["data"], data["labels"], data["groups"]))

# Let's get the permuted labels

In [86]:
PERMUTATION = PermLabels(data["labels"], CV, data["groups"])

# And now run the classification 

In [87]:
# collect data in a dictionary
PERM_SCORES_DICT = {}
for fold, ((train, test), perm) in enumerate(zip(CV, PERMUTATION())):
    print(f"fold number: {fold}")
    print(f"train indices: {train} - test indices: {test}")
    print(
        f"non-permuted train labels: {data['labels'][train]} permuted train labels: {perm}"
    )
    CLF.fit(FMRI_MASKED[train], perm)
    prediction = CLF.predict(FMRI_MASKED[test])
    PERM_SCORES_DICT[f"fold_{fold+1}"] = accuracy_score(
        prediction, data["labels"][test]
    )

fold number: 0
train indices: [18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35] - test indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
non-permuted train labels: [1 2 2 1 2 1 1 1 1 1 2 2 1 2 1 2 2 2] permuted train labels: [2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1]
fold number: 1
train indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17] - test indices: [18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35]
non-permuted train labels: [1 1 2 2 1 1 2 1 1 2 2 2 2 1 2 2 1 1] permuted train labels: [2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2]


In [88]:
PERM_SCORES_DICT

{'fold_1': 0.6111111111111112, 'fold_2': 0.5555555555555556}